#### select target genes

#### pre-process
#### 1. eQTL and sQTL snp-gene
#### 2. Gene annotation (based on TSS (promoter))
#### 3. Hi-C

In [4]:
tissue_dict = {
    "brain":  ["Brain_Frontal_Cortex_BA9", "Brain_Cerebellar_Hemisphere",
               'Brain_Spinal_cord_cervical_c-1','Brain_Cerebellum','Brain_Caudate_basal_ganglia',
               'Brain_Nucleus_accumbens_basal_ganglia','Brain_Hippocampus','Brain_Hypothalamus',
               'Brain_Amygdala','Brain_Putamen_basal_ganglia','Brain_Anterior_cingulate_cortex_BA24',
               'Brain_Substantia_nigra','Brain_Cortex'],
    "muscle": ["Esophagus_Muscularis","Muscle_Skeletal"],
    "liver":  ["Liver"],
    "lung":   ["Lung"],
    "heart":  ['Heart_Left_Ventricle','Heart_Atrial_Appendage'],
    "kidney": ['Kidney_Cortex']
}

In [15]:
#### eQTL or sQTL
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/QTL/GTEx_db/eQTL/GTEx_Analysis_v8_eQTL/")
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/QTL/GTEx_db/sQTL/GTEx_Analysis_v8_sQTL/")
os.chdir("hg38_bed/hg19_bed/")
result_dir = "by_tissue/"
os.system("mkdir -p %s" % result_dir)
#### postfix = ".v8.signif_variant_gene_pairs.txt"
#### file_list = glob.glob("*%s" % postfix)

def map_snp2gene(tissue, bed_list):
    df_list = [pd.read_table(bed, header=None) for bed in bed_list]
    ### combine all samples from the same tissue
    df = pd.concat(df_list)
    df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
    df['gene'] = df.iloc[:,3]
    # snp2gene_dict = dict(zip(df['term'].tolist(), df.iloc[:,3].tolist()))
    #     return snp2gene_dict
    df_sub = df[['term','gene']]
    return df_sub
    
for tissue, p_list in tissue_dict.items():
    print(tissue)
    bed_list = ["%s.bed"%x for x in p_list]
    df = map_snp2gene(tissue, bed_list)  # {"term": "gene"}
    res = os.path.join(result_dir, "%s.bed"%tissue)
    df.to_csv(res, sep="\t", header=False, index=False)

brain
muscle
liver
lung
heart
kidney


In [25]:
#### gene
def extra_tss():
    gene = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/gencode.v19.annotation_Gene.bed"
    df = pd.read_table(gene, header=None)
    df['chr'] = df.iloc[:,0]
    df['start'] = np.where(df.iloc[:,1] > 2000, df.iloc[:,1] - 2000, 0)
    df['end'] = df.iloc[:,2]
    df['gene'] = df.iloc[:,3]
    tss_bed = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/gencode.v19.annotation_GeneTSS2kb-Genend.bed"
    df[['chr','start','end','gene']].to_csv(tss_bed, sep="\t", header=False, index=False)

# extra_tss()

In [26]:
#### TAD
# TAD = "/Charles/project/ASm6A/data/ALS_FUS_mut/GSE77702/TAD_db/TAD_domains_hESC_hg19.bed"

In [30]:
##############################################################
gwas = "/home/database/GWAS/gwas_hg19.bed"
eQTL_dir = "/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/QTL/GTEx_db/eQTL/GTEx_Analysis_v8_eQTL/hg38_bed/hg19_bed/by_tissue/"
sQTL_dir = "/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/QTL/GTEx_db/sQTL/GTEx_Analysis_v8_sQTL/hg38_bed/hg19_bed/by_tissue/"
asm6a_dir = "/Charles/project/ASm6A/ASm6A/by_tissue/"
# asm6a_dir = "/Charles/project/ASm6A/ASm6A/by_tissue/overlap_gwas/"
###########
result_dir = os.path.join(asm6a_dir, "target_genes/")
os.system("mkdir -p %s" % result_dir)
os.chdir(result_dir)
################################################################
feature_list = [sQTL, eQTL, gwas]
name_list = ['sQTL', 'eQTL', 'gwas']

def extract_targetGenes_fromASm6A(asm6a):
    anno="/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/gencode.v19.annotation_GeneTSS2kb-Genend.bed"
    tissue = os.path.basename(asm6a).split(".")[0]
    tmp = "%s_overlap.txt" % tissue
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (asm6a, anno, tmp))
    df = pd.read_table(tmp, sep="\t", header=None)
    target_genes = df.iloc[:,7].tolist()
    os.remove(tmp)
    return list(set(target_genes))

def extract_targetGenes_fromQTL(asm6a, eQTL, sQTL):
    df = pd.read_table(asm6a, header=None)
    df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
    #
    df_eqtl = pd.read_table(eQTL, header=None)
    df_sqtl = pd.read_table(sQTL, header=None)
    df_qtl = pd.concat([df_eqtl, df_sqtl])
    df_qtl.columns = ['term','gene']
    df_com = df.merge(df_qtl, on=['term'], how="left").dropna()
#     print(df_com.head())
    return list(set(df_com['gene'].tolist()))


tissue_list = ['muscle', 'liver', 'lung', 'heart', 'kidney','brain']
for tissue in tissue_list:
    eQTL = os.path.join(eQTL_dir, "%s.txt"%tissue)
    sQTL = os.path.join(sQTL_dir, "%s.txt"%tissue)
    asm6a = os.path.join(asm6a_dir, "%s.bed"%tissue)
    asm6a_genes = extract_targetGenes_fromASm6A(asm6a)
    qtl_genes   = extract_targetGenes_fromQTL(asm6a, eQTL, sQTL)
    target_genes = list(set(asm6a_genes + qtl_genes))
    result_file = "%s_targetGenes.txt"%tissue
    with open(result_file, 'w') as fw:
        fw.writelines(["%s\n"% x for x in target_genes])